# RAG usando [Qdrant](https://qdrant.tech/)

In [1]:
from llama_index.llms.openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

model = "gpt-4o"
llm = OpenAI(model=model, temperature=0)

collection_name = "contracts_info"

question = "O que fala a cláusula 7 do contrato de locação de maquinário?"

contents = []
for file in os.listdir("docs"):
    if file.endswith(".md"):
        with open(f"docs/{file}", "r") as f:
            content = f.read()
            contents.append({"filename": file, "content": content})

## Criação de uma base vetorial para o RAG usando Qdrant

In [2]:
import json
import uuid
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from llama_index.embeddings.openai import OpenAIEmbedding

qdrant_client = QdrantClient(
    url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY")
)
if not qdrant_client.collection_exists(collection_name=collection_name):
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
    )

embedding_model = OpenAIEmbedding(model="text-embedding-ada-002")

for data in contents:
    filename = data["filename"]
    content = data["content"]

    prompt = f"""
        Você é um especialista em leitura e extração de dados em contratos. Seu papel é ler o documento fornecido e separar blocos de texto com base em sua estrutura contratual.

        Regras
            - Não insira mais de uma cláusula dentro de um mesmo bloco;
            - Nos metadados do bloco, sempre informe o filename ({filename});
            - Se o conteúdo do bloco estiver inserido em uma seção ou cláusula identificável, inclua o campo "section" e/ou "clausula" nos metadados;
            - Cada bloco de texto não deve ultrapassar 5.000 caracteres;
            - A resposta deve ser uma lista JSON contendo os campos "metadata" e "content" para cada item;
            - Não adicione comentários, interpretações ou dados externos;
            - Não modifique ou omita nenhuma informação ou caractere do texto original.

        Exemplo de saída:
        [
            {{
                "metadata": {{
                "filename": "teste.docx",
                "section": "IDENTIFICAÇÃO DAS PARTES CONTRATANTES"
                }},
                "content": "**CONTRATO DE TESTE**\\nIDENTIFICAÇÃO DAS PARTES CONTRATANTES\\nCONTRATANTE: (Nome da Contratante), ..."
            }},
            {{
                "metadata": {{
                "filename": "teste.docx",
                "section": "DO OBJETO DO CONTRATO",
                "clausula": "1ª"
                }},
                "content": "Cláusula 1ª. O presente contrato tem como OBJETO, ..."
            }},
            {{
                "metadata": {{
                "filename": "teste.docx",
                }},
                "content": "Por estarem assim justos e contratados, ..."
            }}
        ]

        Documento:
        {content}
    """

    i = 0
    while i < 3:
        try:
            response = llm.complete(prompt).text
            break
        except Exception:
            i += 1

    if "```json" in response:
        response = response.split("```json")[1].split("```")[0].strip()
    elif "```" in response:
        response = response.split("```")[1].split("```")[0].strip()

    chunks_list = json.loads(response)

    points = []
    for chunk in chunks_list:
        metadata = chunk.get("metadata", {})
        content = chunk.get("content", "")
        vector = embedding_model.get_text_embedding(content)

        points.append(
            PointStruct(
                id=str(uuid.uuid4()),
                vector=vector,
                payload={
                    "metadata": metadata,
                    "text": content,
                },
            )
        )

    qdrant_client.upsert(
        collection_name=collection_name,
        points=points,
    )

## RAG com Qdrant

In [ ]:
from qdrant_client import QdrantClient
from llama_index.embeddings.openai import OpenAIEmbedding
from qdrant_client.models import Filter, FieldCondition, MatchValue

qdrant_client = QdrantClient(
    url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY")
)
embedding_model = OpenAIEmbedding(model="text-embedding-ada-002")

# Gera embedding da pergunta
query_vector = embedding_model.get_text_embedding(question)

# Busca documentos similares no Qdrant
results = qdrant_client.query_points(
    collection_name=collection_name,
    query=query_vector,
    limit=10,
)

# Prepara o contexto para a LLM
context = []
for point in results:
    result = {
        "metadata": point[1][0].payload["metadata"],
        "text": point[1][0].payload["text"],
        "score": point[1][0].score,
    }
    context.append(result)

prompt = f"""
    Com base no contexto abaixo, responda à pergunta.

    Contexto:
    {context}

    Pergunta: 
    {question}
"""

response = llm.complete(prompt).text

print(f"Resposta: {response}")

Resposta: A cláusula 7 do contrato de locação de maquinário descreve os deveres da locatária. Ela se compromete a comunicar imediatamente quaisquer ameaças de terceiros contra o maquinário arrendado. Além disso, a locatária deve: 

a) permitir que a locadora fiscalize o maquinário arrendado;  
b) defender a posse e a propriedade das máquinas;  
c) manter sempre um mínimo de três funcionários treinados pela locadora para executar serviços específicos do maquinário;  
d) realizar o pagamento por quaisquer defeitos ou danos causados ao maquinário ou a qualquer uma das máquinas pertencentes ao conjunto.
